In [1]:
%reload_ext autoreload
%autoreload 2
from aldiscore.prediction.extractor import AlDIFeatureExtractor
from Bio.SeqRecord import SeqRecord
import os
from pathlib import Path
from Bio import SeqIO

TEST_FILE_AA = Path("data/aa.fasta")
TEST_FILE_DNA = Path("data/dna.fasta")

sequences = list(SeqIO.parse(TEST_FILE_AA, "fasta"))
print("k =", len(sequences))

k = 4


In [ ]:
extractor = AlDIFeatureExtractor(sequences)
# feat_df = extractor.compute()
ce = extractor._sequence_cross_entropy()
(ce + ce.T) / 2

array([[4.064947 , 4.7289367, 4.3820457, 4.4223604],
       [4.7289367, 3.9773052, 4.3220654, 4.3248262],
       [4.3820457, 4.3220654, 4.1444654, 4.218974 ],
       [4.4223604, 4.3248262, 4.218974 , 4.145658 ]], dtype=float32)